Note: this notebook uses the convention of lowercase Greek characters for spacetime indices, lowercase Latin for spatial, and uppercase Latin for indices on a 2D horizon surface

## Part 1: Introduction and Conceptual Framework

In classical general relativity, the definitions of conserved quantities like mass and angular momentum are intrinsically linked to spacetime symmetries. The mathematical object that describes such a symmetry is a Killing vector field. A vector field, denoted $\xi^a$, is a Killing vector if it generates a continuous isometry, meaning that the metric tensor $g_{ab}$ is unchanged by a flow along $\xi^a$. This condition is expressed by the vanishing of the Lie derivative of the metric with respect to $\xi^\mu$:

$$\mathcal{L}_{\xi} g_{\mu\nu} = \nabla_\mu \xi_\nu + \nabla_\nu \xi_\mu = 0$$
where $\nabla_\mu$ is the covariant derivative compatible with $g_{\mu\nu}$.

The existence of such a vector field implies a conserved quantity along geodesics, providing a rigorous basis for defining the total mass and angular momentum of a system. For stationary, isolated black holes, this approach is perfectly sufficient. The Schwarzschild spacetime, describing a non-rotating black hole, possesses a timelike Killing vector that leads to a conserved energy (mass). The Kerr spacetime, describing a rotating black hole, possesses both a timelike and an axial Killing vector, which allow for the unambiguous definition of both mass and spin angular momentum.

However, this nice formalism breaks down in the context of a binary black hole merger. Such a system is fundamentally dynamic: the black holes inspiral, the spacetime geometry ripples and distorts, and mass-energy is radiated away in the form of gravitational waves. A spacetime that is evolving and radiating is neither stationary nor axisymmetric. Consequently, it does not possess the global symmetries required for the existence of exact Killing vector fields. Any definition of spin that relies on their existence is therefore inapplicable.

This challenge motivates a search for an alternative definition. A natural candidate for the boundary of a black hole is its event horizon—the global, teleological boundary in spacetime separating events that can send signals to a distant observer from those that cannot. However, the event horizon suffers from a critical flaw for practical computation: its location at any given time depends on the *entire future evolution* of the spacetime. It is impossible for a local observer to locate, and it is computationally impossible to determine during a step-by-step numerical simulation that evolves forward in time. This computational impasse was a primary driver for the development of a new, computable, and physically meaningful framework.

### Quasi-Local Horizons

The solution to the breakdown of global definitions is to adopt a quasi-local approach, which defines physical quantities using only information contained within a finite region of spacetime. This method is centered on the concept of marginally outer trapped surfaces (MOTS), which are closed, spacelike 2-surfaces that can be located on a single 3D slice of a numerical simulation. An apparent horizon is typically defined as an outermost MOTS.

**Isolated Horizons (IH): A State of Equilibrium**
An Isolated Horizon provides a quasi-local description of a black hole in equilibrium with its surroundings. It is formally defined as a null 3-hypersurface $H$ on which the expansion of outgoing null rays is zero ($\Theta=0$), and where the gravitational field is stationary *on* the horizon, even if the spacetime far away is dynamic. This idealization allows for the formulation of black hole mechanics without requiring global stationarity. A key condition on the preferred null normal vector $\ell^a$ to the horizon is that the intrinsic geometry of the horizon cross-sections is constant along the flow of $\ell^a$, i.e., $\mathcal{L}_\ell q_{AB} = 0$. This condition ensures that the surface gravity is constant across the horizon, leading to a quasi-local zeroth law of black hole thermodynamics.

**Dynamical Horizons (DH): The Fully General Case**
For a truly evolving black hole, such as during a merger, the Isolated Horizon framework is extended to that of a Dynamical Horizon. A DH is a *spacelike* 3-dimensional manifold that is foliated by MOTS. Unlike an IH, a DH has a net influx of energy and angular momentum, causing its area to increase over time. This framework is powerful because it allows for the derivation of local flux laws that generalize the first and second laws of black hole mechanics. These laws relate the change in the horizon's area, mass, and spin directly to the flux of matter and gravitational radiation crossing the horizon boundary. This provides a direct, computable link between the dynamics of the merger and the properties of the resulting black hole.

The development of this quasi-local formalism was not merely an academic exercise to find an alternative to global definitions. It was a fundamental and necessary evolution of the theory, driven directly by the practical challenges and capabilities of numerical relativity. The inability to compute event horizons in real-time forced a re-evaluation of what defines a black hole from an operational standpoint, leading to a new theoretical structure built around the computable concept of apparent horizons.

### Important Geometric and Physical Quantities

*   **Induced 2-Metric $q_{AB}$**: This is the metric induced on the 2D surface S, which measures distances along the surface. It's computed by projecting the 3D spatial metric $g_{ij}$ (obtained from simulation data) onto the surface.
*   **Surface Covariant Derivative $D_A$**: This is the derivative operator compatible with the induced metric $q_{AB}$, allowing for derivatives on the curved 2D surface.
*   **Horizon Distortion**: The distortion of the horizon's geometry by a vector field $\phi^a$ is measured by the Lie derivative of the induced metric, $\mathcal{L}_\phi q_{AB}$. For a true Killing vector, this quantity would be zero. In the quasi-local framework, the algorithm seeks to find a $\phi^a$ that minimizes the integrated magnitude of this distortion, thereby finding the "best-fit" symmetry.
*   **Rotational 1-form $\omega_A$**: This 1-form is a crucial quantity that encodes the rotational properties of the spacetime geometry on the horizon, representing the effects of frame-dragging. In the isolated horizon formalism, it is defined via the derivative of the null normal, $D_A \ell^b = \omega_A \ell^b$. For computational purposes, it is calculated from the extrinsic curvature $K_{ab}$ of the 3D slice and the intrinsic metric $q_{AB}$.
*   **Approximate Killing Vector (AKV) $\phi^A$**: This is the vector field on the horizon surface that best approximates a rotational symmetry. Finding this vector by minimizing the distortion functional is the central computational challenge of the spin calculation algorithm.

## Part 2: Algorithmic Plan



**Table 1: Key Mathematical Objects for Spin Calculation**

| Symbol | Name | Definition (in terms of 3D quantities) | Physical Interpretation |
| :--- | :--- | :--- | :--- |
| $g_{ab}$ | 3D Spatial Metric | Grid function from simulation | Measures distances on the 3D spatial slice. |
| $K_{ab}$ | Extrinsic Curvature | Grid function from simulation | Measures how the 3D slice is curved within the 4D spacetime. |
| $S$ | Horizon Surface | Set of points $h(\theta, \phi)$ from BH@HAHA | The 2D boundary of the black hole on a given slice. |
| $s^a$ | Surface Normal | $\frac{\nabla_a r}{\|\nabla r\|}$ | Unit vector normal to $S$, lying within the 3D slice. |
| $q_{AB}$ | Induced 2-Metric | $\partial_A x^a \partial_B x^b g_{ab}$ | Measures distances on the horizon surface $S$. |
| $D_A$ | Surface Covariant Derivative | Covariant derivative compatible with $q_{AB}$ | Derivative operator restricted to the 2D surface. |
| $\omega_A$ | Rotational 1-form | $s^j (K_{ja} - q_{ja} K)$ | Encodes the "twist" or rotational frame-dragging on the horizon. |
| $\phi^A$ | Approx. Killing Vector | Vector field on $S$ that minimizes $\int\|\mathcal{L}_\phi q_{AB}\|^2 dA$ | The best-fit rotational symmetry axis on the distorted horizon. |

### Step 1: Horizon Data Extraction and Preparation

The first step is to get all the surface data from BH@HAHA and the necessary spacetime data (probably from the simulation itself)

*   **Horizon Shape Data**: `BH@HAHA` produces ASCII data files, typically named like `h.t<timestep>.ah<horizon_num>.gp`, which describe the shape of the horizon. These files contain a set of Cartesian points on the surface, but might need to be reorganized/shaped by their angular coordinates ($\theta$, $\phi$). I believe a function from the visualization script should already do most of this.
*   **3D Grid Function Data**: The simulation evolves the spacetime geometry, storing the results on 3D grids. The essential values for this analysis are the components of the 3D spatial metric, $g_{ab}$ (e.g., `gxx`, `gxy`, etc.), the extrinsic curvature, $K_{ab}$ (e.g., `kxx`, `kxy`, etc.), and the induced metric $q_{AB}$ (e.g., `qthetatheta`, `qthetaphi`, etc.).
*   **Interpolation**: This is a critical linking step. The metric and extrinsic curvature data exist on the vertices of a 3D computational grid, while the spin calculation requires their values *on* the 2D horizon surface. Therefore, the 3D grid functions must be interpolated to the $(x, y, z)$ locations of the horizon surface points extracted in the previous step. This procedure yields discrete sets of values for $g_{ab}$ and $K_{ab}$ at each point on the horizon surface.

### Step 2: Finding the Approximate Axial Symmetry Vector

Because a dynamic horizon is distorted and lacks a perfect symmetry, the core of the algorithm is to find a vector field that is "as close as possible" to being a true rotational Killing vector.

*   **The Variational Principle**: The method seeks a vector field $\phi^A$ tangent to the horizon surface that minimizes the integrated squared "distortion" it induces on the horizon's intrinsic geometry. This distortion is measured by the Lie derivative of the intrinsic metric, $\mathcal{L}_\phi q_{AB}$. The functional to be minimized is:
    $$I[\phi] = \oint_S |\mathcal{L}_\phi q_{AB}|^2 dA$$
*   **Simplification via a Scalar Potential**: To make this problem easier to solve computationally, the vector field $\phi^A$ is assumed to be divergence-free on the 2-sphere, $D_A \phi^A = 0$. This is a property of Killing vectors on a 2-sphere and is a reasonable approximation here. This assumption allows $\phi^A$ to be expressed as the surface "curl" of a scalar potential $\zeta$:
    $$\phi^A = \epsilon^{AB} D_B \zeta$$
    where $\epsilon^{AB}$ is the Levi-Civita tensor (area form) on the 2-surface.
*   **The Generalized Eigenvalue Problem**: Substituting the potential $\zeta$ into the functional $I[\phi]$ and performing a variational minimization leads to a generalized eigenvalue problem for $\zeta$. This problem takes the form:
    $$A \zeta = \lambda B \zeta$$
    Here, $A$ and $B$ are second-order, linear, elliptic differential operators on the sphere, whose explicit forms are constructed from the intrinsic metric $q_{AB}$ and its derivatives. The eigenvector $\zeta$ corresponding to the smallest non-trivial eigenvalue $\lambda$ provides the potential for the desired approximate Killing vector. This procedure is typically carried out three times to find three approximately orthogonal rotational vectors corresponding to the global $x$, $y$, and $z$ axes of the simulation.


### Step 3: Spin Vector and Christodoulou-Ruffini Mass Calculation

Once the approximate Killing vector field $\phi^a$ has been determined, the angular momentum is computed using an integral derived from the isolated and dynamical horizon formalisms:

$$J = \frac{1}{8\pi} \oint_S \omega_B \phi^B dA$$

The terms in this integral are:
*   $\omega_A$: The rotational 1-form, computed at each point on the horizon surface using the interpolated values of $g_{ab}$ and $K_{ab}$.
*   $\phi^A$: The approximate Killing vector field found in Step 2.
*   $dA$: The differential surface area element, given by $dA = \sqrt{\det(q_{AB})} d\theta d\phi$, which is also computed from the interpolated metric data.

The integral is evaluated numerically. This involves discretizing the horizon surface, calculating the value of the integrand $\omega_B \phi_a^B$ at each discrete point, multiplying by the area element $\Delta A$ of the corresponding surface patch, and summing the results over the entire surface. A good example can be found in `bah_diagnostics_area_centroid_and_Theta_norms()`.

Once we have the magnitude of $J$ we can use it to scale the components of the AKV, leaving us with a quasi-local spin vector.

$J$ also goes into the Christodoulou-Ruffini formula for the mass-energy of a rotating BH, which uses the irreducible/areal mass of the BH, also calculated by BH@HAHA based on the surface area of the AH. The formula is below and should be straightforward to implement:

$$M^2 = M_{irr}^2 + \frac{J^2}{4M_{irr}^2}$$ 

## Part 3: Execution and Visualization

This functionality will be added via a Python/NRPy file quasi_local_mass_spin.py, using function `register_CFunction_diagnostics_quasi_local_mass_spin()`. This will generate a C function file that performs the actual calculations.

The ability to visualize spin is already included in the new_vis_styles.py script of the `bh_vis` repo. All that's needed is a Cartesian vector, which should be easy to output from BH@HAHA itself or convert from spherical coordinates in the Python script.